In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import glob
import matplotlib.pyplot as plt 
from collections import deque 
from tensorflow.keras import layers
import time 
import scipy.io as sio

from vehicle_model_variant import Environment 
from cell_model import CellModel 
from driver_MDP import Driver_MDP 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
# env = Environment(cell_model, drving_cycle, battery_path, motor_path, 10)
driver = Driver_MDP(0.02)

num_states = 4

In [3]:
class OUActionNoise: 
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None): 
        self.theta = theta 
        self.mean = mean 
        self.std_dev = std_deviation 
        self.dt = dt 
        self.x_initial = x_initial 
        self.reset() 
        
    def reset(self): 
        if self.x_initial is not None: 
            self.x_prev = self.x_initial 
        else: 
            self.x_prev = 0 
            
    def __call__(self): 
        x = (
             self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt 
            + self.std_dev * np.sqrt(self.dt) * np.random.normal() 
        )
        self.x_prev = x 
        return x 

In [4]:
class Buffer: 
    def __init__(self, buffer_capacity=100000, batch_size=64):      
        self.buffer_capacity = buffer_capacity 
        self.batch_size = batch_size 
        self.buffer_counter = 0 
        
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, 1))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))
        
    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity 

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        
        self.buffer_counter += 1 
        
    def learn(self): 
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)

        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        
        with tf.GradientTape() as tape: 
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.square(y - critic_value)) 
        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables) 
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )
        
        with tf.GradientTape() as tape: 
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            actor_loss = - tf.math.reduce_mean(critic_value)
        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables) 
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )
        

In [5]:
def update_target(tau): 
    new_weights = [] 
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_critic.set_weights(new_weights)
    
    new_weights = [] 
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_actor.set_weights(new_weights)
    

In [6]:
def get_actor(): 
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
    
    inputs = layers.Input(shape=(num_states))
    inputs_batchnorm = layers.BatchNormalization()(inputs)
    
    out = layers.Dense(512, activation="relu")(inputs_batchnorm)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="sigmoid", 
                          kernel_initializer=last_init)(out)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def get_critic(): 
    state_input = layers.Input(shape=(num_states))
    state_input_batchnorm = layers.BatchNormalization()(state_input)
    
    state_out = layers.Dense(16, activation="relu")(state_input_batchnorm)
#     state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(32, activation="relu")(state_out)
#     state_out = layers.BatchNormalization()(state_out)
    
    action_input = layers.Input(shape=(1))
    action_out = layers.Dense(32, activation="relu")(action_input)
#     action_out = layers.BatchNormalization()(action_out)
    
    concat = layers.Concatenate()([state_out, action_out]) 
    
    out = layers.Dense(512, activation="relu")(concat)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)
    
    model = tf.keras.Model([state_input, action_input], outputs)
    return model 
    

In [8]:
def policy(state, noise_object): 
    j_min = state[0][2].numpy()
    j_max = state[0][3].numpy()
    sampled_action = tf.squeeze(actor_model(state)) 
    noise = noise_object()
    sampled_action = sampled_action.numpy() + noise 
    legal_action = sampled_action * j_max 
    legal_action = np.clip(legal_action, j_min, j_max)
#     print(j_min, j_max, legal_action, noise)
    return legal_action 
    

In [9]:
def policy_epsilon_greedy(state, eps): 
    j_min = state[0][-2].numpy()
    j_max = state[0][-1].numpy()

    if random.random() < eps: 
        a = random.randint(0, 9)
        return np.linspace(j_min, j_max, 10)[a]
    else: 
        sampled_action = tf.squeeze(actor_model(state)).numpy()  
        legal_action = sampled_action * j_max 
        legal_action = np.clip(legal_action, j_min, j_max)
        return legal_action

In [10]:
std_dev = 0.2 
ou_noise = OUActionNoise(mean=0, std_deviation=0.2)

critic_lr = 0.0005 
actor_lr = 0.00025 
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 200 
gamma = 0.95 
tau = 0.001 

MAX_EPSILON = 1.0 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
DELAY_TRAINING = 5000 

In [11]:
def initialization(): 
    actor_model = get_actor() 
    critic_model = get_critic() 
    target_actor = get_actor() 
    target_critic = get_critic() 
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())
#     actor_model.load_weights("./DDPG1_trial1/actor_model_checkpoint")
#     critic_model.load_weights("./DDPG1_trial1/critic_model_checkpoint")
#     target_actor.load_weights("./DDPG1_trial1/target_actor_checkpoint")
#     target_critic.load_weights("./DDPG1_trial1/target_critic_checkpoint")
    
    buffer = Buffer(500000, BATCH_SIZE)
    return actor_model, critic_model, target_actor, target_critic, buffer

In [12]:
def save_weights(actor_model, critic_model, target_actor, target_critic, root): 
    if not os.path.exists(root): 
        os.makedirs(root)
        
    actor_model.save_weights("./{}/actor_model.h5".format(root))
    critic_model.save_weights("./{}/critic_model.h5".format(root))
    target_actor.save_weights("./{}/target_actor.h5".format(root))
    target_critic.save_weights("./{}/target_critic.h5".format(root))
    print("model is saved..")

In [13]:
def initialization_env(driving_path, reward_factor):
    env = Environment(cell_model, driving_path, battery_path, motor_path, reward_factor)
    return env 

In [14]:
def test_agent(actor_model, reward_factor):
#     test_cycle = driver.get_cycle() 
    test_cycle_path = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
    test_cycle = sio.loadmat(test_cycle_path)
    test_cycle = test_cycle["sch_cycle"][:, 1]
    env = initialization_env(test_cycle, reward_factor)
    
    total_reward = 0
    state = env.reset() 
    while True: 
        tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
        action = policy_epsilon_greedy(tf_state, -1)
        next_state, reward, done = env.step(action)
        
        state = next_state 
        total_reward += reward 
        
        if done: 
            break 
        
    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
    
    print("******************* Test is start *****************")
#     print(test_cycle)
    print('Total reward: {}'.format(total_reward), 
          "SOC: {:.4f}".format(env.SOC), 
          "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
          "Fuel Consumption: {:.4f}".format(env.fuel_consumption))
    print("******************* Test is done *****************")
    print("")
    plt.subplot(1, 2, 1)
    plt.plot(test_cycle)
    plt.subplot(1, 2, 2)
    plt.plot(env.history["Action"])
    plt.show() 
    return env.history  
    

In [15]:
# print(env.version)

# num_trials = 1
reward_factors = [7, 8, 9, 10]
results_dict = {} 
driving_cycle_path = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
driving_cycle = sio.loadmat(driving_cycle_path)
driving_cycle = driving_cycle["sch_cycle"][:, 1]

for reward_factor in reward_factors: 
    print("")
    print("Trial {}".format(reward_factor))
    print("")
    
    actor_model, critic_model, target_actor, target_critic, buffer = initialization()
    
    eps = MAX_EPSILON 
    steps = 0
    
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    episode_test_history = [] 
    episode_num_test = [] 
    for ep in range(total_episodes): 
#         driving_cycle = driver.get_cycle() 
        env = initialization_env(driving_cycle, reward_factor)
        
        start = time.time() 
        state = env.reset() 
        episodic_reward = 0 

        while True: 
            tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            action = policy_epsilon_greedy(tf_state, eps)
    #         print(action)
            next_state, reward, done = env.step(action)
            if done: 
                next_state = [0] * num_states 

            buffer.record((state, action, reward, next_state))
            episodic_reward += reward 

            if steps > DELAY_TRAINING: 
                buffer.learn() 
                update_target(tau)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * (steps
                                                                        -DELAY_TRAINING))

            steps += 1

            if done: 
                break 

            state = next_state 

        elapsed_time = time.time() - start 
        print("elapsed_time: {:.3f}".format(elapsed_time))
        episode_rewards.append(episodic_reward) 
        episode_SOCs.append(env.SOC)
        episode_FCs.append(env.fuel_consumption) 

    #     print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
        SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
        print(
              'Episode: {}'.format(ep + 1),
              "Exploration P: {:.4f}".format(eps),
              'Total reward: {}'.format(episodic_reward), 
              "SOC: {:.4f}".format(env.SOC), 
              "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
              "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
        )
        print("")
        
        if (ep + 1) % 10 == 0: 
#             history = test_agent(actor_model, reward_factor)
            history = env.history 
            episode_test_history.append(history) 
            episode_num_test.append(ep + 1)
            
#         if (ep + 1) % 200 == 0:             
    root = "DDPG_cycleOne_reward_factor{}".format(reward_factor)
    save_weights(actor_model, critic_model, target_actor, target_critic, root)
            
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs, 
        "test_history": episode_test_history, 
        "test_episode_num": episode_num_test, 
    }


Trial 7

maximum steps, simulation is done ... 
elapsed_time: 9.446
Episode: 1 Exploration P: 1.0000 Total reward: -3537.6415759272327 SOC: 1.0000 Cumulative_SOC_deviation: 478.6676 Fuel Consumption: 186.9685

maximum steps, simulation is done ... 
elapsed_time: 9.534
Episode: 2 Exploration P: 1.0000 Total reward: -3583.7075839103486 SOC: 0.9999 Cumulative_SOC_deviation: 484.9024 Fuel Consumption: 189.3908

maximum steps, simulation is done ... 
elapsed_time: 9.174
Episode: 3 Exploration P: 1.0000 Total reward: -3632.6107087301025 SOC: 1.0000 Cumulative_SOC_deviation: 491.8252 Fuel Consumption: 189.8341


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the aut

maximum steps, simulation is done ... 
elapsed_time: 86.614
Episode: 28 Exploration P: 0.5172 Total reward: -610.447839379491 SOC: 0.7546 Cumulative_SOC_deviation: 67.4387 Fuel Consumption: 138.3769

maximum steps, simulation is done ... 
elapsed_time: 88.892
Episode: 29 Exploration P: 0.5034 Total reward: -1073.9038336371468 SOC: 0.8278 Cumulative_SOC_deviation: 132.9929 Fuel Consumption: 142.9536

maximum steps, simulation is done ... 
elapsed_time: 87.308
Episode: 30 Exploration P: 0.4901 Total reward: -713.8030257373596 SOC: 0.7691 Cumulative_SOC_deviation: 82.1286 Fuel Consumption: 138.9026

maximum steps, simulation is done ... 
elapsed_time: 88.204
Episode: 31 Exploration P: 0.4771 Total reward: -467.14698729854325 SOC: 0.6317 Cumulative_SOC_deviation: 48.5163 Fuel Consumption: 127.5330

maximum steps, simulation is done ... 
elapsed_time: 86.243
Episode: 32 Exploration P: 0.4644 Total reward: -844.2427003307353 SOC: 0.6021 Cumulative_SOC_deviation: 102.6095 Fuel Consumption: 12

C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment_essential_modelChange\DDPG_rewardfactor_comparison_final\vehicle_model_variant.py:251: RuntimeWarning: invalid value encountered in double_scalars
  del_i = (1 / (2 * r_cha)) * (v_cha - (v_cha ** 2 - 4 * r_cha * p_bat) ** (0.5)) * (p_bat < 0) + (1 / (


maximum steps, simulation is done ... 
elapsed_time: 80.219
Episode: 46 Exploration P: 0.3194 Total reward: -3419.069927615135 SOC: 0.4894 Cumulative_SOC_deviation: 470.8948 Fuel Consumption: 122.8067

maximum steps, simulation is done ... 
elapsed_time: 80.052
Episode: 47 Exploration P: 0.3110 Total reward: -180.07960002899995 SOC: 0.6153 Cumulative_SOC_deviation: 8.3698 Fuel Consumption: 121.4908

maximum steps, simulation is done ... 
elapsed_time: 80.108
Episode: 48 Exploration P: 0.3028 Total reward: -179.4393571430983 SOC: 0.6090 Cumulative_SOC_deviation: 8.3253 Fuel Consumption: 121.1621

maximum steps, simulation is done ... 
elapsed_time: 80.052
Episode: 49 Exploration P: 0.2949 Total reward: -173.86732768549598 SOC: 0.5982 Cumulative_SOC_deviation: 7.6981 Fuel Consumption: 119.9803

maximum steps, simulation is done ... 
elapsed_time: 79.966
Episode: 50 Exploration P: 0.2872 Total reward: -193.77791245426988 SOC: 0.6149 Cumulative_SOC_deviation: 10.2881 Fuel Consumption: 121.

maximum steps, simulation is done ... 
elapsed_time: 107.690
Episode: 87 Exploration P: 0.1104 Total reward: -282.5834149059805 SOC: 0.5842 Cumulative_SOC_deviation: 23.9886 Fuel Consumption: 114.6631

maximum steps, simulation is done ... 
elapsed_time: 107.418
Episode: 88 Exploration P: 0.1076 Total reward: -260.8614083727572 SOC: 0.5903 Cumulative_SOC_deviation: 20.9927 Fuel Consumption: 113.9123

maximum steps, simulation is done ... 
elapsed_time: 107.644
Episode: 89 Exploration P: 0.1050 Total reward: -374.1221542501641 SOC: 0.5518 Cumulative_SOC_deviation: 37.7654 Fuel Consumption: 109.7647

maximum steps, simulation is done ... 
elapsed_time: 108.144
Episode: 90 Exploration P: 0.1024 Total reward: -279.07862081970893 SOC: 0.5595 Cumulative_SOC_deviation: 24.0632 Fuel Consumption: 110.6365

maximum steps, simulation is done ... 
elapsed_time: 108.273
Episode: 91 Exploration P: 0.0999 Total reward: -283.07190820822274 SOC: 0.5883 Cumulative_SOC_deviation: 24.4728 Fuel Consumption

...

maximum steps, simulation is done ... 
elapsed_time: 69.236
Episode: 20 Exploration P: 0.6418 Total reward: -3229.758010082754 SOC: 0.9997 Cumulative_SOC_deviation: 307.6826 Fuel Consumption: 152.9317

maximum steps, simulation is done ... 
elapsed_time: 69.167
Episode: 21 Exploration P: 0.6247 Total reward: -3419.5866021161814 SOC: 0.9982 Cumulative_SOC_deviation: 326.4964 Fuel Consumption: 154.6222

maximum steps, simulation is done ... 
elapsed_time: 69.209
Episode: 22 Exploration P: 0.6080 Total reward: -2628.270594205328 SOC: 0.9994 Cumulative_SOC_deviation: 247.3999 Fuel Consumption: 154.2717

maximum steps, simulation is done ... 
elapsed_time: 69.276
Episode: 23 Exploration P: 0.5918 Total reward: -2431.2763613764027 SOC: 0.9418 Cumulative_SOC_deviation: 228.1200 Fuel Consumption: 150.0764

maximum steps, simulation is done ... 
elapsed_time: 69.337
Episode: 24 Exploration P: 0.5761 Total reward: -2726.394010576031 SOC: 0.9633 Cumulative_SOC_deviation: 257.4112 Fuel Consumption

maximum steps, simulation is done ... 
elapsed_time: 69.758
Episode: 61 Exploration P: 0.2149 Total reward: -210.50542727789102 SOC: 0.5906 Cumulative_SOC_deviation: 9.4875 Fuel Consumption: 115.6303

maximum steps, simulation is done ... 
elapsed_time: 70.018
Episode: 62 Exploration P: 0.2094 Total reward: -213.03667796449307 SOC: 0.5865 Cumulative_SOC_deviation: 9.9041 Fuel Consumption: 113.9954

maximum steps, simulation is done ... 
elapsed_time: 70.764
Episode: 63 Exploration P: 0.2040 Total reward: -213.75681884621648 SOC: 0.5995 Cumulative_SOC_deviation: 9.8135 Fuel Consumption: 115.6217

maximum steps, simulation is done ... 
elapsed_time: 70.517
Episode: 64 Exploration P: 0.1987 Total reward: -229.31185745228532 SOC: 0.5913 Cumulative_SOC_deviation: 11.6269 Fuel Consumption: 113.0430

maximum steps, simulation is done ... 
elapsed_time: 69.776
Episode: 65 Exploration P: 0.1936 Total reward: -234.97044774485383 SOC: 0.5845 Cumulative_SOC_deviation: 12.2057 Fuel Consumption: 112

maximum steps, simulation is done ... 
elapsed_time: 73.565
Episode: 102 Exploration P: 0.0765 Total reward: -200.90216667401918 SOC: 0.5932 Cumulative_SOC_deviation: 9.2340 Fuel Consumption: 108.5627

maximum steps, simulation is done ... 
elapsed_time: 74.258
Episode: 103 Exploration P: 0.0747 Total reward: -180.68552630878702 SOC: 0.5930 Cumulative_SOC_deviation: 7.1603 Fuel Consumption: 109.0828

maximum steps, simulation is done ... 
elapsed_time: 73.811
Episode: 104 Exploration P: 0.0729 Total reward: -173.07649863251333 SOC: 0.5952 Cumulative_SOC_deviation: 6.3705 Fuel Consumption: 109.3718

maximum steps, simulation is done ... 
elapsed_time: 70.024
Episode: 105 Exploration P: 0.0712 Total reward: -189.12999548524158 SOC: 0.5934 Cumulative_SOC_deviation: 8.0468 Fuel Consumption: 108.6615

maximum steps, simulation is done ... 
elapsed_time: 70.014
Episode: 106 Exploration P: 0.0696 Total reward: -206.81619379415847 SOC: 0.5917 Cumulative_SOC_deviation: 9.8654 Fuel Consumption: 

maximum steps, simulation is done ... 
elapsed_time: 70.424
Episode: 143 Exploration P: 0.0316 Total reward: -235.1702472266271 SOC: 0.5927 Cumulative_SOC_deviation: 12.7005 Fuel Consumption: 108.1650

maximum steps, simulation is done ... 
elapsed_time: 70.787
Episode: 144 Exploration P: 0.0310 Total reward: -219.86547466316074 SOC: 0.5921 Cumulative_SOC_deviation: 11.2133 Fuel Consumption: 107.7321

maximum steps, simulation is done ... 
elapsed_time: 70.311
Episode: 145 Exploration P: 0.0304 Total reward: -235.0374192143273 SOC: 0.5908 Cumulative_SOC_deviation: 12.7290 Fuel Consumption: 107.7469

maximum steps, simulation is done ... 
elapsed_time: 70.662
Episode: 146 Exploration P: 0.0299 Total reward: -256.0377233769655 SOC: 0.5913 Cumulative_SOC_deviation: 14.8161 Fuel Consumption: 107.8765

maximum steps, simulation is done ... 
elapsed_time: 70.602
Episode: 147 Exploration P: 0.0293 Total reward: -249.19025651023043 SOC: 0.5947 Cumulative_SOC_deviation: 14.0441 Fuel Consumption

maximum steps, simulation is done ... 
elapsed_time: 71.008
Episode: 184 Exploration P: 0.0170 Total reward: -246.9429955034999 SOC: 0.5982 Cumulative_SOC_deviation: 13.7009 Fuel Consumption: 109.9342

maximum steps, simulation is done ... 
elapsed_time: 71.159
Episode: 185 Exploration P: 0.0168 Total reward: -239.0574434933148 SOC: 0.5957 Cumulative_SOC_deviation: 12.8433 Fuel Consumption: 110.6241

maximum steps, simulation is done ... 
elapsed_time: 70.876
Episode: 186 Exploration P: 0.0166 Total reward: -208.13111019771017 SOC: 0.5918 Cumulative_SOC_deviation: 9.8947 Fuel Consumption: 109.1846

maximum steps, simulation is done ... 
elapsed_time: 70.633
Episode: 187 Exploration P: 0.0164 Total reward: -211.56831725925397 SOC: 0.5974 Cumulative_SOC_deviation: 10.1143 Fuel Consumption: 110.4256

maximum steps, simulation is done ... 
elapsed_time: 70.902
Episode: 188 Exploration P: 0.0163 Total reward: -188.84974235996356 SOC: 0.5990 Cumulative_SOC_deviation: 7.8529 Fuel Consumption:

In [16]:
with open("DDPG_cycleOne_7to10.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [17]:
# results_dict